##### EDA

In [2]:
import torchxrayvision as xrv

C:\Program Files\Python312\Lib\site-packages\torchxrayvision\utils.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
d = xrv.datasets.COVID19_Dataset(imgpath="data/images/", csvpath="data/csv/metadata.csv")

In [4]:
img = d[0]['img'][0]
print(f"sample image of shape {img.shape}")
print(f"dataset of length {len(d)}")
print(img)

sample image of shape (888, 882)
dataset of length 535
[[1024.      1015.9686  1007.93726 ... -469.83527 -477.86664 -477.86664]
 [1024.      1024.      1015.9686  ... -493.92938 -493.92938 -493.92938]
 [1024.      1024.      1015.9686  ... -501.96075 -509.99213 -509.99213]
 ...
 [1024.      1024.       983.84314 ... -172.6745  -156.61176 -148.58038]
 [1024.      1024.       983.84314 ... -252.98822 -244.95685 -244.95685]
 [1024.      1024.       983.84314 ... -437.70978 -429.6784  -421.64703]]


##### Dataloader

In [1]:
from dataLoader import *
from torch.utils.data import DataLoader

C:\Users\Hoo Kai Sng\OneDrive - National University of Singapore\Documents\NUS\CS3244\Project\xrayenv\Lib\site-packages\torchxrayvision\utils.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
train_dataset = Covid19DataSet(transform = ToTensor(), train = True)
test_dataset = Covid19DataSet(transform = ToTensor(), train = False)

Shape of X_train is (321, 1, 224, 224). Shape of ytrain is (321, 1)
Shape of X_test is (214, 1, 224, 224). Shape of ytest is (214, 1)
Shape of X_train is (321, 1, 224, 224). Shape of ytrain is (321, 1)
Shape of X_test is (214, 1, 224, 224). Shape of ytest is (214, 1)


In [3]:
batch_size = 16
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

In [4]:
train_iter = iter(train_loader)
test_iter = iter(test_loader)

In [5]:
train_samples, train_labels = next(train_iter)
test_samples, test_labels = next(test_iter)
print(train_samples.shape, train_labels.shape, test_samples.shape, test_labels.shape)

torch.Size([16, 1, 224, 224]) torch.Size([16, 1]) torch.Size([16, 1, 224, 224]) torch.Size([16, 1])


##### Building of model

In [ ]:
from models import *

In [149]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 5)
        self.pool = nn.MaxPool2d(4, 4)
        self.conv2 = nn.Conv2d(8, 16, 5)

        self.fc1 = nn.Linear(16 * 12 * 12, 576)
        self.fc2 = nn.Linear(576, 128)
        self.fc3 = nn.Linear(128, 1)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 12 * 12)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train_model(model, train_loader, epochs, learning_rate = 0.001, criterion = nn.BCEWithLogitsLoss()):
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    
    print("Training model...")
    model.train()
    for epoch in range(epochs):
        total_loss, count = 0, 0
        for i, (images, labels) in enumerate(train_loader):
            count += 1
            # forward pass
            y_preds = model(images)
            loss = criterion(y_preds, labels)
            total_loss += loss.item()

            # backward prop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        avg_loss = total_loss / count 
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")
        
    print("Done!")
    

def evaluate_model(model, test_loader):
    with torch.no_grad():
        tp = tn = fp = fn = 0
        for image, label in test_loader:
            output = model(image)

            y_pred = (torch.sigmoid(output) >= 0.5).float()
            n = len(y_pred)
            for i in range(n):
                pred_val, label_val = y_pred[i].item(), label[i].item()
                tp += (pred_val == label_val == 1)
                tn += (pred_val == label_val == 0)
                fp += (pred_val == 1 and label_val == 0)
                fn += (pred_val == 0 and label_val == 1)
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    print(f"Precision: {precision} . Recall: {recall}")



In [123]:
model = ConvNet()

##### Training Model

In [124]:
epochs = 10
train_model(model, train_loader, epochs)

Training model...
Epoch [1/10], Loss: 3.1032
Epoch [2/10], Loss: 0.6451
Epoch [3/10], Loss: 0.5790
Epoch [4/10], Loss: 0.5633
Epoch [5/10], Loss: 0.5075
Epoch [6/10], Loss: 0.4489
Epoch [7/10], Loss: 0.4802
Epoch [8/10], Loss: 0.3598
Epoch [9/10], Loss: 0.3577
Epoch [10/10], Loss: 0.2929
Done!


##### Evaluation

In [150]:
evaluate_model(model, test_loader)

Precision: 0.6838709677419355 . Recall: 0.8153846153846154
